In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns
from sklearn import datasets

데이터 전처리

우리가 실제로 접하게 되는 데이터는 사이킷런에서 제공하는 데이터와는 다르게 머신러닝을 적용하기 전에 데이터를 가공해야 하는 경우가 대부분이다. 이를 데이터 전처리 또는 피쳐 엔지니어링 등으로 부른다.

결측치(missing value) 처리

데이터 셋의 일부에 데이터가 존재하지 않을 경우 결측치라고 부르며, 머신러닝 알고리즘을 적용하기 전에 미리 결측치를 처리하는 것이 좋다.

In [2]:
# 데이터프레임에 데이터가 존재하지 않는 결측치는 없지만 부적절한 값(이상치)이 존재한다.
# 나이에 1000이라는 값이 존재하는 것과 나머지 피쳐에 'unknown'이라는 값이 입력된 부적절한 값은 결측치로 처리하는 것이 좋다.
df = pd.DataFrame([
    [42, 'male', 12, 'reading', 'class2'],
    [35, 'unknown', 3, 'cooking', 'class1'],
    [1000, 'female', 7, 'cycling', 'class3'],
    [1000, 'unknown', 21, 'unknown', 'unknown']
])
# 나이, 성별, 대어난 달, 취미, 레이블
df.columns = ['age', 'gender', 'month_birth', 'hobby', 'label']
df

,age,gender,month_birth,hobby,label
0,42,male,12,reading,class2
1,35,unknown,3,cooking,class1
2,1000,female,7,cycling,class3
3,1000,unknown,21,unknown,unknown


In [3]:
# unique() 메소드로 각 시리즈의 유일한 값을 확인할 수 있다.
print('age:', df.age.unique())
print('gender:', df.gender.unique())
print('month_birth:', df.month_birth.unique())
print('hobby:', df.hobby.unique())
print('label:', df.label.unique())

age: [  42   35 1000]
gender: ['male' 'unknown' 'female']
month_birth: [12  3  7 21]
hobby: ['reading' 'cooking' 'cycling' 'unknown']
label: ['class2' 'class1' 'class3' 'unknown']


In [4]:
# 부적절한 데이터 값을 결측치로 바꾸기 위해서 np.nan을 사용한다.
df.loc[df.age > 150, ['age']] = np.nan
df.loc[df.gender == 'unknown', ['gender']] = np.nan
df.loc[df.month_birth > 12, ['month_birth']] = np.nan
df.loc[df.hobby == 'unknown', ['hobby']] = np.nan
df.loc[df.label == 'unknown', ['label']] = np.nan
df

,age,gender,month_birth,hobby,label
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3
3,NaN,NaN,NaN,NaN,NaN


In [5]:
# 데이터 셋의 각 열의 결측치 개수를 확인한다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          2 non-null      float64
 1   gender       2 non-null      object 
 2   month_birth  3 non-null      float64
 3   hobby        3 non-null      object 
 4   label        3 non-null      object 
dtypes: float64(2), object(3)
memory usage: 288.0+ bytes


In [6]:
df.isnull().sum()

age            2
gender         2
month_birth    1
hobby          1
label          1
dtype: int64

결측치를 처리하는 데 주로 쓰이는 방법은 결측치를 삭제하거나 다른 값으로 변경하는 방법이 있다.

In [7]:
# dropna() 메소드로 결측치를 삭제할 수 있다.
# axis 속성을 생략하거나 0으로 지정하면 결측치가 하나라도 포함된 모든 행을 삭제한다.
# inplace 속성을 True로 지정하면 실행 결과를 대입하지 않아도 실행 결과를 즉시 반영한다.
df.dropna() # df.dropna(axis=0)

,age,gender,month_birth,hobby,label
0,42.0,male,12.0,reading,class2


In [8]:
# axis 1으로 지정하면 결측치가 하나라도 포함된 모든 열을 삭제한다.
df.dropna(axis=1)

""
0
1
2
3


In [9]:
# dropna() 메소드의 how 속성을 'all'로 지정하면 모든 값이 결측치인 행을 삭제한다.
df.dropna(how='all')

,age,gender,month_birth,hobby,label
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [10]:
# dropna() 메소드의 thresh 속성에 숫자를 지정하면 결측치를 제외한 값의 개수가 숫자보다 작은 행을 삭제한다.
df.dropna(thresh=1)

,age,gender,month_birth,hobby,label
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [11]:
# dropna() 메소드의 subset 속성에 열이름을 지정하면 특정 열에 결측치가 존재하는 행을 삭제한다.
df.dropna(subset=['gender'])

,age,gender,month_birth,hobby,label
0,42.0,male,12.0,reading,class2
2,NaN,female,7.0,cycling,class3


In [12]:
# fillna() 메소드로 결측치를 대체할 값을 지정할 수 있다.
df.fillna(0)

,age,gender,month_birth,hobby,label
0,42.0,male,12.0,reading,class2
1,35.0,0,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,0,0.0,0,0


In [13]:
# fillna() 메소드의 value 속성으로 결측치를 대체할 값이 저장된 딕셔너리를 지정해서 열별로 결측치를
# 대체할 값을 지정할 수 있다.
# 열 이름을 key로 하고 결측치를 대체할 값을 value로 하는 딕셔너리를 지정한다.
alter_values = {'age': 0, 'gender': 'U', 'month_birth': 0, 'hobby': 'N', 'label': 'class4'}
df.fillna(value=alter_values, inplace=True)
df

,age,gender,month_birth,hobby,label
0,42.0,male,12.0,reading,class2
1,35.0,U,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,U,0.0,N,class4


기계 처리에 적합하도록 클래스 레이블 설정

label 열을 보면 문자열(object) 형태인데, 기계 처리에 적합하도록 하기 위해 문자열을 상수(int)로 바꿔야 하는 경우가 있다.

사이킷런을 사용하지 않고 레이블 설정하기

In [14]:
# 라벨링할 레이블만 얻어와서 오름차순으로 정렬한다.
label = df.label.values
print(label)
label.sort() # 오름차순 정렬
# label = label[::-1] # 내림차순 정렬
print(label)

['class2' 'class1' 'class3' 'class4']
['class1' 'class2' 'class3' 'class4']


In [15]:
# 반복문을 실행해서 정렬된 레이블을 key로 일련번호를 value로 하는 딕셔너리를 만든다.
label_dict = {}
for value, key in enumerate(label):
    # print(value, key)
    label_dict[key] = value
label_dict

{'class1': 0, 'class2': 1, 'class3': 2, 'class4': 3}

In [16]:
# replace() 메소드를 이용해서 딕셔너리에 저장된 데이터로 레이블을 라벨링한다.
df['label_1'] = df.label.replace(label_dict)
df

,age,gender,month_birth,hobby,label,label_1
0,42.0,male,12.0,reading,class1,0
1,35.0,U,3.0,cooking,class2,1
2,0.0,female,7.0,cycling,class3,2
3,0.0,U,0.0,N,class4,3


사이킷런으로 레이블 설정하기

In [17]:
# 레이블 라벨링을 하기 위해 import 한다.
from sklearn.preprocessing import LabelEncoder

In [18]:
le = LabelEncoder() # 라벨링을 실행할 객체를 만든다.
label = df.label.values # 라벨링을 실행할 데이터를 얻어온다.
print(label)
# le.fit(label) # 학습시킨다. 라벨링한다.
# new_label = le.transform(label) # 학습한 결과를 적용시킨다.
# fit_transform() 메소드는 학습(fit)과 적용(transform)을 한 번에 실행한다.
new_label = le.fit_transform(label) # 학습 후 적용시킨다.
print(new_label)
# 라벨링한 결과를 데이터프레임에 넣어준다.
df['label_2'] = new_label
df

['class1' 'class2' 'class3' 'class4']
[0 1 2 3]


,age,gender,month_birth,hobby,label,label_1,label_2
0,42.0,male,12.0,reading,class1,0,0
1,35.0,U,3.0,cooking,class2,1,1
2,0.0,female,7.0,cycling,class3,2,2
3,0.0,U,0.0,N,class4,3,3


원-핫 인코딩(One-Hot Encoding)

원-핫 인코딩은 클래스를 라벨링하는 또 다른 방법으로 오직 0과 1만 사용한 벡터를 이용해서 데이터 값을 나타내는 것이다.

판다스를 이용한 원-핫 인코딩

In [19]:
pd.get_dummies(df.label)

,class1,class2,class3,class4
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


In [20]:
# 위 방법에서는 길이가 4인 벡터를 이용해서 클래스를 구분했다.
# drop_first 속성값을 True로 지정하면 벡터 길이를 하나 줄인 벡터를 이용해 원-핫 인코딩을 할 수 있다.
# drop_first=True 속성을 지정하면 3개의 0과 1의 조합으로 4개의 클래스가 구분되는 것을 확인할 수 있다.
pd.get_dummies(df.label, drop_first=True)

,class2,class3,class4
0,0,0,0
1,1,0,0
2,0,1,0
3,0,0,1


사이킷런을 이용한 원-핫 인코딩

In [21]:
# 원-핫 인코딩을 하기 위해 import 한다.
from sklearn.preprocessing import OneHotEncoder

In [22]:
ohe = OneHotEncoder() # 원-핫 인코딩을 실행할 객체를 만든다.
label = df[['label']] # 원-핫 인코딩을 실행할 데이터를 얻어온다.
print(label)
# ohe.fit(label) # 학습시킨다. 원-핫 인코딩을 실행한다.
# new_label = ohe.transform(label) # 학습한 결과를 적용시킨다.
new_label = ohe.fit_transform(label) # 학습 후 적용시킨다.
print(type(new_label))
print(new_label) # 결과를 출력하면 희소 행렬로 출력된다.
print(type(new_label.toarray()))
print(new_label.toarray())

    label
0  class1
1  class2
2  class3
3  class4
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
<class 'numpy.ndarray'>
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


텐서플로우를 이용한 원-핫 인코딩

In [23]:
# 원-핫 인코딩을 하기 위해 import 한다.
from tensorflow.keras.utils import to_categorical

In [24]:
label = df[['label_2']] # 원-핫 인코딩을 실행할 데이터를 얻어온다.
new_label = to_categorical(label) # 원-핫 인코딩을 실행한다. 인수로 반드시 숫자를 넘겨야 한다.
print(type(new_label))
print(new_label)

<class 'numpy.ndarray'>
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


데이터 스케일링(data scaling) - 정규화

피쳐는 제각기 다른 단위를 가지기 때문에 이로 인해 숫자 자체의 크기 차이가 발생한다. 데이터 스케일링은 데이터 값이 단위의 영향을 받지 않도록 변형하는 것을 의미한다. 전체 데이터를 학습 데이터와 테스트 데이터로 나누고 스케일링의 기준이 되는 값을 구할 때는 학습 데이터만으로 구한다.

표준화(standard) 스케일링

대표적은 데이터 스케일링 방법으로 정규 분포(평균이 0, 표준편차 1)를 따르도록 변경하는 방법이다.

$$\frac{x_i - \bar x}{\sigma}$$

위 식의 의미는 기존 데이터 값에서 평균을 뺀 후 표준편차로 나눈다는 의미이다.

In [25]:
# 표준화 스케일링을 하기 위해 import 한다.
from sklearn.preprocessing import StandardScaler

In [26]:
std = StandardScaler() # 표준화 스케일링 객체를 만든다.
y = df[['month_birth']]
print(y)
# std.fit(y) # 학습시킨다. 표준화 스케일러를 실행한다.
# x_std = std.transform(y) # 학습한 결과를 적용시킨다.
x_std = std.fit_transform(y) # 학습 후 적용시킨다.
print(x_std)

   month_birth
0         12.0
1          3.0
2          7.0
3          0.0
[[ 1.44444444]
 [-0.55555556]
 [ 0.33333333]
 [-1.22222222]]


In [27]:
# 실제로 표준화 스케일링 후 데이터가 평균이 0, 표준편차가 1인지 확인한다.
print('평균: {:.1f}, 표준편차: {:.1f}'.format(np.mean(x_std), np.std(x_std)))

평균: -0.0, 표준편차: 1.0


로버스트(robust) 스케일링

표준화 스케일링을 변형한 방법으로 중앙값(median)과 사분위수(quantile)를 사용한다. 극단값의 영향을 거의 받지 않는다는 장점이 있다.

$$\frac{x_i - q_2}{q_3 - q_1}$$

위 식에서 q<sub>1</sub>은 1사분위수 q<sub>2</sub>는 2사분위수(중위수), q<sub>3</sub>은 3사분위수를 의미한다.

In [28]:
# 로버스트 스케일링을 하기 위해 import 한다.
from sklearn.preprocessing import RobustScaler

In [29]:
robust = RobustScaler() # 로버스트 스케일링 객체를 만든다.
y = df[['month_birth']]
# robust.fit(y) # 학습시킨다. 로버스트 스케일러를 실행한다.
# x_robust = robust.transform(y) # 학습한 결과를 적용시킨다.
x_robust = robust.fit_transform(y) # 학습 후 적용시킨다.
print(x_robust)

[[ 1.16666667]
 [-0.33333333]
 [ 0.33333333]
 [-0.83333333]]


최소-최대(min-max) 스케일링

데이터 값의 범위를 최소값(0)과 최대값(1) 사이의 값으로 범위를 제한한다.

$$\frac{x_i - min(x)}{max(x) - min(x)}$$

In [30]:
# 최대-최소 스케일링을 하기 위해 import 한다.
from sklearn.preprocessing import MinMaxScaler

In [31]:
minmax = MinMaxScaler() # 최대-최소 스케일링 객체를 만든다.
y = df[['month_birth']]
# minmax.fit(y) # 학습시킨다. 최대-최소 스케일러를 실행한다.
# x_minmax = minmax.transform(y) # 학습한 결과를 적용시킨다.
x_minmax = minmax.fit_transform(y) # 학습 후 적용시킨다.
print(x_minmax)

[[1.        ]
 [0.25      ]
 [0.58333333]
 [0.        ]]


노멀(normalizer) 스케일링

벡터의 유클리드 거리(두 점 사이의 거리 계산에 사용)가 1이 되도록 데이터 값을 변경한다. 주로 벡터의 길이는 상관없고 방향(각도)만 고려할 때 사용한다.  
표준화, 로버스트, 최소-최대 스케일링은 열 기준인 것과 다르게 노멀 스케일링은 행 기준이다.

$$\frac{x_i}{\sqrt{x_i^2 + y_i^2 + z_i^2}}$$

In [32]:
# 노멀 스케일링을 하기 위해 import 한다.
from sklearn.preprocessing import Normalizer

In [33]:
normal = Normalizer() # 노멀 스케일링 객체를 만든다.
y = df[['age', 'month_birth']]
# normal.fit(y) # 학습시킨다. 노멀 스케일러를 실행한다.
# x_normal = normal.transform(y) # 학습한 결과를 적용시킨다.
x_normal = normal.fit_transform(y) # 학습 후 적용시킨다.
print(x_normal)

[[0.96152395 0.27472113]
 [0.99634665 0.08540114]
 [0.         1.        ]
 [0.         0.        ]]


데이터 스케일링 과정에서 fit() 메소드는 학습 데이터에 대해서만 사용하며, 테스트 데이터에는 fit() 메소드를 사용하지 않고 transform() 메소드만 사용한다.  
그 이유는 스케일러는 학습 데이터를 기반으로 만들어야 하기 때문이다. 만약에 학습 데이터와 테스트 데이터에 fit() 메소드를 실행해서 각각의 데이터로 스케일링한다면  학습 데이터와 테스트 데이터의 분포가 다르기 때문에 학습 데이터와 테스트 데이터의 스케일링 범위 및 파라미터 달라진다.

from sklearn.preprocessing import StandardScaler  
std = StandardScaler()

학습 데이터는  
std.fit(x_train) # 학습 데이터로 스케일러를 만든다.  
x_train_std = std.transform(x_train) # 학습 데이터에 스케일러를 적용시킨다.  
또는  
x_train_std = std.fit_transform(x_train)  
방법을 사용해서 스케일러를 학습시킨 후 적용한다.

테스트 데이터는 fit() 메소드를 사용하지 않고 학습 데이터로 학습된 스케일러에 테스트 데이터를 적용만 시킨다.  
x_test_std = std.transform(x_test)